#### Test:

In [42]:
import dataset
from xai_cola import data,ml_model,counterfactual_explainer,cola,counterfactual_refiner
from xai_cola.cola_explainer import COunterfactualwithLimitedActions

In [43]:
%reload_ext autoreload
%autoreload 2

In [44]:
dataset1 =dataset.GermanCreditDataset()
df1 = dataset1.get_dataframe()
df1.head(2)

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
0,67,1,2,1,0,1,1169,6,5,0
1,22,0,2,1,1,2,5951,48,5,1


In [45]:
# dataset2 = dataset.HotelBookingsDataset()
# df2 = dataset2.get_dataframe()
# df2.head(2)

##### Configuration

In [46]:
data1 = data.PandasData(data=df1, target_name="Risk")

In [47]:
import joblib

In [48]:
lgbmcClassifier = joblib.load('LGBMClassifier.pkl')
print(f'----LGBMClassifier.pkl model has been loaded----')

----LGBMClassifier.pkl model has been loaded----


In [49]:
ml_model1 = ml_model.Model(model=lgbmcClassifier, backend='sklearn')  

In [50]:
counterfactual_explainer1 = counterfactual_explainer.DiCE(ml_model=ml_model1, data=data1, sample_num=4)  #will add more input params according to the dice_ml algorithm 

---- predictions have been made----


100%|██████████| 4/4 [00:00<00:00,  6.64it/s]


---- predictions have been made----


100%|██████████| 4/4 [00:00<00:00,  6.98it/s]


In [51]:
x_factual, x_counterfactual = counterfactual_explainer1.generate_counterfactuals()   #numpy类型

---- predictions have been made----


100%|██████████| 4/4 [00:00<00:00,  7.00it/s]


##### *choose policy and make limitation


In [52]:
ot_pshap_max = COunterfactualwithLimitedActions(
    data=data1, 
    ml_model=ml_model1, 
    explainer=counterfactual_explainer1,  # use explainer  / factual & counterfactual
    x_factual=None,
    x_counterfactual=None,
    limited_actions=10, 
    matcher="ot", # joint_probability
    attributor="pshap", # used to get the varphi
    Avalues_method="max" # get the q
    )

In [54]:
factual, ce, ace = ot_pshap_max.generate_results()

---- predictions have been made----
---- predictions have been made----
---- predictions have been made----


In [55]:
factual

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
0,36,1,2,2,1,1,7297,60,0,1
1,24,0,3,1,2,2,7408,60,1,1
2,33,1,1,1,1,1,2579,12,1,1
3,36,1,2,1,1,1,1372,12,1,1


In [56]:
ce

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
0,33,1,1,1,3,1,2579,38,1,0
1,36,1,2,1,1,0,1372,12,4,0
2,35,0,3,1,2,2,7408,60,1,0
3,36,1,2,2,1,1,7297,46,0,0


In [57]:
ace

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
0,36,1,2,2,1,1,7297,46,0,0
1,35,0,3,1,2,2,7408,60,1,0
2,33,1,1,1,3,1,2579,38,1,0
3,36,1,2,1,1,0,1372,12,4,0
